In [52]:
import import_ipynb
from maze_common import Maze, shortestPathSearch
import time
import random as rnd
import numpy as np
import copy

In [59]:
maze = Maze(5, .3)
solvable = maze.isSolvable()
print(maze.board, "\nIs solvable? ", solvable)

[[ 0  0 -1 -1  0]
 [ 0  0 -1  0  0]
 [ 0  0 -1  0  0]
 [ 0  0  0  0 -1]
 [-1  0  0  0  0]] 
Is solvable?  True


In [49]:
def manhattanDistance(maze, cell):
    return abs(cell.row - (maze.dim - 1)) + abs(cell.col - (maze.dim - 1))

if (solvable):
    startTime = time.time()
    shortestPath = shortestPathSearch(maze, heuristicFunction = manhattanDistance)
    print("***** Found shortest path in %s seconds *****" % (time.time() - startTime))
    print(shortestPath)
    

***** Found shortest path in 0.0049877166748046875 seconds *****
[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4)]


In [62]:
def thinMaze(maze, fractionRemove):
    numRemove = round(fractionRemove * maze.obstacles.shape[0])
    thinMaze = copy.deepcopy(maze)
    for i in range(0,numRemove):
        indexRemove = int(rnd.random() * thinMaze.obstacles.shape[0])
        obstacleX, obstacleY = thinMaze.obstacles[indexRemove]
        thinMaze.obstacles = np.delete(thinMaze.obstacles,indexRemove, axis=0)
        thinMaze.board[obstacleX][obstacleY] = 0
    return thinMaze
    
thinnedMaze = thinMaze(maze, .6)
print("Original obstacles: ", maze.obstacles.shape[0], "\n", maze.obstacles)
print("Thinned obstacles: ", thinnedMaze.obstacles.shape[0], "\n",thinnedMaze.obstacles)
print("Thinned maze:\n", thinnedMaze.board)

Original obstacles:  6 
 [[0 2]
 [0 3]
 [1 2]
 [2 2]
 [3 4]
 [4 0]]
Thinned obstacles:  2 
 [[0 3]
 [3 4]]
Thinned maze:
 [[ 0  0  0 -1  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0 -1]
 [ 0  0  0  0  0]]


In [75]:
visited = {}

def thinnedMazeShortestPathLength(maze, cell):
    shortestPathLength = 0
    if ((cell.row, cell.col)) not in visited:
        shortestPath = shortestPathSearch(thinnedMaze, startCoords = (cell.row, cell.col))
        shortestPathLength = len(shortestPath)
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cell.row, cell.col)]
    return shortestPathLength

if (solvable):
    startTime = time.time()
    shortestPath = shortestPathSearch(maze, heuristicFunction = thinnedMazeShortestPathLength)
    print("***** Found shortest path in %s seconds *****" % (time.time() - startTime))
    print(shortestPath)

***** Found shortest path in 49.16647791862488 seconds *****
[(0, 0), (1, 0), (1, 1), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3), (4, 4)]
